# GeoAlchemy2 ORM Test -- Import ODHF

Source: [https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html](https://geoalchemy-2.readthedocs.io/en/latest/orm_tutorial.html)

In [ ]:
# read configuration
import psycopg2, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

#conn_string = "host=%(SERVER)s user=%(USERNAME)s dbname=%(DATABASE)s password=%(PASSWORD)s sslmode=%(SSLMODE)s" % config
#conn = psycopg2.connect(conn_string) 
#print("Connection established")


# Some psycopg2 intialization

from register_adapters import register_adapters

register_adapters()

## Connect to the DB

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql+psycopg2://%(USERNAME)s:%(PASSWORD)s@%(SERVER)s/%(DATABASE)s?sslmode=%(SSLMODE)s" % config,
    client_encoding="utf8", 
    echo=True
)

## Declare a Mapping

We will try to map the CSD digital boundary files 

In [ ]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float
from geoalchemy2 import Geometry

# TODO: import models from external file
# references:
# - https://docs.pylonsproject.org/projects/pyramid_cookbook/en/latest/database/sqlalchemy.html#importing-all-sqlalchemy-models
# - https://stackoverflow.com/questions/9088957/sqlalchemy-cannot-find-a-class-name

#from ORM_StandardGeographicClassification import CSD
from sqlalchemy.orm import relationship, backref

Base = declarative_base()

class CSD(Base):
    __tablename__ = "CSD_DBF_2021"
    DGUID = Column(String, primary_key=True)
    CSDUID = Column(Integer)
    CSDNAME = Column(String)
    CSDTYPE = Column(String)
    LANDAREA = Column(Float)
    geometry = Column(Geometry(geometry_type='GEOMETRY', srid=4326))

class EducationalFacility(Base):
    __tablename__ = "ODHF_2_1"
    __table_args__ = {'extend_existing': True}
    index = Column(String, primary_key=True)
    Facility_Name = Column(String)
    Full_Addr = Column(String)     
    Unit = Column(String)         
    Street_No = Column(String)    
    Street_Name = Column(String)     
    City = Column(String)           
    Prov_Terr = Column(String)      
    Postal_Code = Column(String)
    geometry = Column(Geometry(geometry_type='POINT', srid=4326))
    CSD = relationship(
        'CSD',
        primaryjoin='func.ST_Contains(foreign(CSD.geometry), EducationalFacility.geometry).as_comparison(1, 2)',
        #backref=backref('Educational_Facility', uselist=False),
        viewonly=True,
        uselist=False,

    )

## Create the Table in the Database

In [ ]:
try:
    EducationalFacility.__table__.drop(engine)
except:
    pass

EducationalFacility.__table__.create(engine)

In [ ]:
import pandas as pd
import os, zipfile

zf = zipfile.ZipFile(os.path.join(os.path.abspath(""), "data", "ODEF_v2.1_EN.zip")) 
df = pd.read_csv(zf.open('ODEF_v2.1_EN/ODEF_v2_1.csv'), encoding="cp1252",keep_default_na=False)

df.Full_Addr = df.Full_Addr.astype(str)
df.Unit = df.Unit.astype(str)
df.Street_No = df.Street_No.astype(str)
df.Street_Name = df.Street_Name.astype(str)
df.City = df.City.astype(str)
df.Prov_Terr = df.Prov_Terr.astype(str)
df.Postal_Code = df.Postal_Code.astype(str)

print(df.info())

df = df[(df.Latitude != "..") & (df.Longitude != "..")]

In [ ]:
df.Facility_Type.unique()

## Create a Session

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()


In [ ]:
# helper function to create table entries from GeoDataFrame
def insert_edu(session, x):

    row = EducationalFacility( 
                index = x.Index,
                Facility_Name = x.Facility_Name.strip(),
                Full_Addr = x.Full_Addr.strip(),   
                Unit = x.Unit.strip(),      
                Street_No = x.Street_No.strip(),   
                Street_Name = x.Street_Name.strip(),
                City = x.City.strip(),           
                Prov_Terr = x.Prov_Terr.strip(),    
                Postal_Code = x.Postal_Code.strip(),
                geometry = 'POINT(%s %s)' % (x.Longitude, x.Latitude)
                )


    session.add(row)


session.rollback()

df.apply(lambda x: insert_edu(session, x), axis=1)

# commit changes
session.commit()

## Query

In [ ]:
# Get all schools
school_list = session.query(EducationalFacility).order_by(EducationalFacility.Facility_Name).all()

for i in school_list[:10]:
    print(i.Facility_Name, i.Full_Addr, i.CSD.CSDUID, i.CSD.CSDNAME)


In [ ]:
csd_list = session.query(CSD).order_by(CSD.CSDNAME).where(CSD.CSDTYPE=="IRI").all()

for i in csd_list:
    print(i.CSDNAME)